In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import time
from splinter import Browser
import pandas as pd

In [2]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "/Users/thomasadamson/Downloads/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

## NASA Mars News

In [3]:
#Start browser session for mars.nasa.gov/news/
browser = init_browser()

url = "https://mars.nasa.gov/news/"
browser.visit(url)

time.sleep(1)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [4]:
#Use class 'content_title to retrieve article title'
title = soup.find_all('div', class_= 'content_title')[1].text
title

'Another First: Perseverance Captures the Sounds of Driving on Mars'

In [5]:
#Use class 'article_teaser_body to retrieve article tag'
tag = soup.find_all('div', class_='article_teaser_body')[0].text
tag

'NASA’s newest rover recorded audio of itself crunching over the surface of the Red Planet, adding a whole new dimension to Mars exploration.'

In [6]:
browser.quit()

## JPL Mars Space Images - Featured Image

In [7]:
#Start browser session for mars.nasa.gov/news/
browser = init_browser()

url = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/"
browser.visit(url + "index.html")

time.sleep(1)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [8]:
#Use img tag and class to find pathing to image
image_path = soup.find('img', class_ = 'headerimage')['src']

In [9]:
#Full url includes both the initial page and the pathing
featured_image_url = url + image_path

In [10]:
featured_image_url

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg'

In [11]:
browser.quit()

## Mars Facts

In [12]:
#Define url for pandas scrape
url= 'https://space-facts.com/mars/'

In [13]:
tables = pd.read_html(url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [14]:
mars_facts = tables[0]
mars_facts

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [15]:
mars_facts.rename(columns={0: 'Measure', 1: 'Value'}, inplace=True)
mars_facts.set_index('Measure', inplace=True)

In [16]:
mars_facts_html = mars_facts.to_html()
mars_facts_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Measure</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n  <

## Mars Hemispheres

In [17]:
#Start browser session 
browser = init_browser()

url = "https://astrogeology.usgs.gov"
browser.visit(url + "/search/results?q=hemisphere+enhanced&k1=target&v1=Mars")

time.sleep(1)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [18]:
hemispheres = soup.find_all('div', class_ = 'item')

In [19]:
#append each hemisphere's link title into a list so we can click them in splinter
pages = []
for title in hemispheres:
    pages.append(title.find('h3').text)
    
pages

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [20]:
#Loop through each page and get image url

img_urls = []

for image_page in pages:
    
    #Declare the starting url for each loop iteration
    url = "https://astrogeology.usgs.gov"
    browser.visit(url + "/search/results?q=hemisphere+enhanced&k1=target&v1=Mars")
    
    #Click on the appropriate link
    browser.links.find_by_partial_text(image_page).click()
    
    #Set a working html paser for the new page
    html = browser.html
    soup = bs(html, "html.parser")
    
    #Extract the imag path
    img_path = soup.find('img', class_='wide-image')['src']
    
    full_url = url + img_path
    
    img_urls.append(full_url)
    
img_urls

['https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg']

In [21]:
#Make a list of dictioinaries using the pages and urls lists
hemisphere_info = []

for x in range(len(pages)):
    temp_dict = {'title': pages[x], 'img_url': img_urls[x]}
    
    hemisphere_info.append(temp_dict)
    
hemisphere_info

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]